In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# OpenAI API를 활용한 텍스트-음성 변환 (TTS) 튜토리얼 (2025년 3월 기준)

OpenAI의 강력한 API를 이용하면 입력한 텍스트를 자연스러운 음성으로 변환할 수 있습니다. 이 튜토리얼에서는 Jupyter Notebook에서 실행할 수 있는 Python 코드와 함께 OpenAI의 텍스트-음성 변환(Text-to-Speech, TTS) 기능을 단계별로 설명합니다. 실습을 통해 API 키 설정부터 기본적인 TTS 사용법, 고급 기능 활용, 그리고 생성된 오디오를 저장하고 재생하는 방법까지 알아보겠습니다.

### 주요 학습 내용:
1. 환경 설정: OpenAI API 키를 안전하게 저장하고 로드하는 방법 (python-dotenv 활용)
2. 기본 TTS 변환: 간단한 텍스트를 음성으로 변환하는 예제 코드
3. 고급 기능 활용: 음성 모델 선택, 음성 목소리(voice) 변경, 언어 지원 및 속도 조절 등 추가 기능
4. 오디오 파일 저장 및 재생: 생성된 음성을 파일로 저장하고 Jupyter Notebook에서 재생하는 방법
각 섹션에서는 관련 코드 블록과 함께 자세한 설명을 제공하니, 코드를 직접 실행해보며 따라 할 수 있습니다.

## 1. 환경 설정

Python 코드에서 python-dotenv로 .env 파일을 불러온 뒤, openai.OpenAI() 클래스를 이용해 API 클라이언트 인스턴스를 생성합니다. 이때 API 키는 명시적으로 전달하거나 환경 변수 OPENAI_API_KEY가 설정되어 있어야 합니다. 환경 변수 또는 인자로 API 키를 지정하지 않으면 OpenAI 라이브러리는 다음과 같은 오류를 발생시킵니다:

> OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

아래는 환경 로드 및 클라이언트 생성 예제입니다:


In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os
load_dotenv()
client = OpenAI()

위 코드에서는 .env에서 불러온 API 키로 client 객체를 생성했습니다. 이제 이 client를 통해 DALL-E를 비롯한 OpenAI API 요청을 보낼 수 있습니다. (참고로, api_key를 생략하면 OPENAI_API_KEY 환경 변수를 자동으로 참조합니다.)

## 2. 기본 TTS 변환: 텍스트를 음성으로 간단히 만들기

환경 설정이 완료되었다면, 이제 텍스트를 음성으로 변환해보겠습니다. OpenAI의 TTS API는 입력 텍스트를 받아 사람이 말하는 것 같은 음성 오디오 데이터를 반환합니다. 기본적으로 **tts-1**이라는 모델을 사용하며, 음성의 종류(voice)를 선택할 수 있습니다. 우선 간단한 예제로 기본 모델과 한 가지 목소리를 이용해 동작을 확인해보겠습니다.


In [3]:
# 변환할 텍스트
text = "Hello, OpenAI! This is a text to speech test."
with client.audio.speech.with_streaming_response.create(
    model = "tts-1",
    voice = "nova",
    input = text
) as response:
    response.stream_to_file('data/ch05_output.mp3')

In [4]:
from IPython.display import Audio
Audio('data/ch05_output.mp3', autoplay=True)

In [5]:
text = "Hello, OpenAI! This is a text to speech test."
response = client.audio.speech.create(
    model = "tts-1",
    voice = "nova",
    input = text
)

In [9]:
response.stream_to_file('data/ch05_output.mp3')

C:\Users\901-00\AppData\Local\Temp\ipykernel_7524\704624324.py:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file('data/ch05_output.mp3')


위 코드에서는 openai_client.audio.speech.create(...) 메소드를 호출하여 TTS 요청을 보냈습니다. 주요 파라미터는 다음과 같습니다:
- model="tts-1": OpenAI의 기본 TTS 모델을 지정합니다. tts-1 모델은 실시간 응답에 최적화된 모델로, 비교적 빠르게 결과를 반환합니다.
- voice="nova": 출력 음성의 목소리를 지정합니다. OpenAI TTS에는 여러 프리셋 목소리가 있는데, 여기서는 "nova"라는 목소리를 선택했습니다. (목소리 종류에 대해서는 다음 섹션에서 자세히 다룹니다.)
- input=text: 변환할 텍스트 문자열을 입력으로 전달합니다.

create 메소드를 호출하면 OpenAI API에 요청을 보내고, 음성 오디오 데이터를 응답으로 받습니다. response 객체에는 이진 형식의 오디오 데이터가 포함되어 있습니다. print(type(response))로 객체 타입을 출력해보면, OpenAI 라이브러리의 내부 응답 객체임을 확인할 수 있습니다. 

이 response로부터 실제 오디오 바이너리 데이터를 추출하려면 response.content 속성을 사용할 수 있습니다. 예를 들어:


In [12]:
audio_data = response.content
print(f'생성된 오디어 데이터 크기 : {len(audio_data)}byte')
with open('data/ch05_nova.mp3', 'wb') as audio_file:
    audio_file.write(audio_data)

생성된 오디어 데이터 크기 : 53280byte


In [15]:
Audio('data/ch05_nova.mp3', autoplay=True) # 주피터노트북에서 오디오 듣기
# import os # vscode에서 오디오 듣기
# os.startfile('data/ch05_nova.mp3')

위와 같이 response.content를 사용하면 음성 데이터의 바이너리를 얻을 수 있고, len(audio_data)로 바이트 크기를 확인할 수 있습니다. 이제 텍스트가 음성 데이터로 변환된 것을 확인했으니, 다음 단계에서 다양한 고급 기능을 활용하는 방법을 알아보겠습니다.

주의: 이 시점까지는 메모리 상에 음성 데이터가 있을 뿐, 실제 소리로 들어보지는 않았습니다. response.content를 파일로 저장하거나 플레이어를 통해 재생해야 비로소 음성을 들을 수 있습니다. 오디오 데이터 저장과 재생 방법은 튜토리얼 뒷부분(섹션 4)에서 다룹니다.

## 3. 고급 기능 활용: 모델 선택, 목소리 변경, 언어 지원 등

OpenAI의 TTS API는 기본 기능 외에도 다양한 옵션을 제공합니다. 이 섹션에서는 고급 기능으로 다음을 다뤄봅니다:
- 다양한 목소리(voice) 선택: 남성/여성 및 톤이 다른 여러 프리셋 음성을 사용할 수 있습니다.
- 모델 선택: 기본 모델 tts-1 외에 고품질 모델 tts-1-hd 사용법
- 다국어 지원: 영어 이외의 언어로도 TTS가 가능 (한국어, 스페인어 등)
- 음성 속도 조절: speed 파라미터로 발음 속도를 빠르게 또는 느리게 조절

각 기능별로 어떻게 사용되는지 예제를 통해 살펴보겠습니다.

### 3.1 여러 가지 음성 목소리 선택하기
OpenAI TTS에는 다양한 **프리셋 목소리(voice)**가 준비되어 있어 텍스트를 여러 스타일의 음성으로 들을 수 있습니다. 초기에는 6가지 목소리를 제공하며, 각 목소리는 고유한 음색과 톤을 가지고 있습니다. 제공되는 대표적인 음성 이름은 다음과 같습니다:
- Alloy – (예상: 남성적이고 부드러운 음색)
- Echo – (맑고 청아한 톤)
- Fable – (동화 구연 같은 따뜻한 톤)
- Nova – (친근한 여성 음색으로 추정)
- Onyx – (깊고 진중한 남성 음색)
- Shimmer – (밝고 생동감 있는 여성 음색)

각 목소리는 영어를 기반으로 최적화되어 있지만, 다국어 텍스트도 발음할 수 있습니다. 원하는 분위기나 성별 느낌에 따라 위 이름들 중 선택하면 됩니다. 사용 방법은 간단히 voice 파라미터에 해당 이름 문자열을 지정하는 것입니다. 예를 들어, 앞서 기본 예제에서는 "nova" 음성을 사용했는데, 다른 음성으로 바꾸어 출력해보겠습니다:


In [ ]:
# 다른 목소리로 TTS 변환
text = "This text will be spoken with a different voice."
response_diff_voice = client